In [1]:
#数据准备
from __future__ import print_function
import numpy as np
import xarray as xr
import netCDF4 as nc
import pandas as pd
import math
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from metpy import units
from netCDF4 import Dataset
import metpy.constants as constants
import metpy.calc as mpcalc
import cartopy.io.shapereader as shpreader
from matplotlib.path import Path
from cartopy.mpl.patch import geos_to_path
from metpy.interpolate import inverse_distance_to_grid
from scipy.interpolate import griddata
from scipy.interpolate import Rbf
import scipy.ndimage as ndimage
from mpl_toolkits.basemap import Basemap
import time
from matplotlib.dates import DateFormatter
import cftime
import mpath
import matplotlib.path as mpath
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import os
import glob
import dask.array as da
import cmaps
import import_ipynb
import code_study as cs
# %run code_study.ipynb
# from _future_ import print_function
import torch
import code_study as cs
from datetime import datetime as dt
import cdsapi

In [2]:
def time_before_after(date_str):
    from datetime import datetime, timedelta

    # 将字符串转换为日期对象
    date_obj = datetime.strptime(date_str, '%Y%m%d')
    
    # 计算前一天的日期
    previous_day = date_obj - timedelta(days=1)
    # if str(date_str)[4:]=='0101':
    #     previous_day = date_obj + timedelta(days=0)    
    # 计算后一天的日期
    next_day = date_obj + timedelta(days=1)
    # if str(date_str)[4:]=='0228':
    #     next_day = date_obj + timedelta(days=0) 
    # 将日期对象格式化为字符串
    previous_day_str = previous_day.strftime('%Y%m%d')
    next_day_str = next_day.strftime('%Y%m%d')
    if str(date_str)[4:]=='0101':
        return date_str,next_day_str
    if str(date_str)[4:]=='0228':
        return previous_day_str,date_str
    else:
        return previous_day_str,date_str,next_day_str

In [3]:
def data_download(var,date):
    c = cdsapi.Client()
    
    # The directory for forecastsd
    ## Use os.path.join to give cross platform compatibility
    
    # The variables required
    variables_all = ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure','geopotential', 'temperature', 'u_component_of_wind', 'v_component_of_wind', "relative_humidity"]
    var_all=['T2m', 'U10', 'V10', 'MSL','Z', 'T', 'U', 'V', 'R']
    var_index=var_all.index(var)
    # Area to download
    area = [90, 0, -90, 359]
    
    # Pressure levels required
    pressure_levels = [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]
    
    # Download the surface data
    c.retrieve('reanalysis-era5-single-levels', {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': variables_all[var_index],
        'date': date,
        'time': [
            "00:00",
            "06:00",
            "12:00",
            "18:00",
        ],
        'area': area,
        "grid": "1.0/1.0",
    },  ("E:\data\ERA5.daily.1x1\\"+var_all[var_index]+"\Hourly\\"+ var_all[var_index]+date+'.nc'))
    
    # # Download the upper air data
    # c.retrieve('reanalysis-era5-pressure-levels', {
    #     'product_type': 'reanalysis',
    #     'format': 'netcdf',
    #     'variable': variables_all[var_index],
    #     'pressure_level': pressure_levels,
    #     'date': date,
    #     'time': [
    #         "00:00",
    #         "06:00",
    #         "12:00",
    #         "18:00",
    #     ],
    #     'area': area,
    #     "grid": "1.0/1.0",
    # },  ("E:\data\ERA5.daily.1x1\\"+var_all[var_index]+"\Hourly\\"+ var_all[var_index]+date+'.nc'))


In [ ]:
date='20231201'
var='T2m'
for i in range(61+366+92):
    date_following = pd.to_datetime(date) + pd.Timedelta(days=i)
    date_str = date_following.strftime('%Y%m%d')
    if os.path.exists(r'E:\data\ERA5.daily.1x1\\'+var+'\\Hourly\\'+var+date_str+'.nc') == True:
        continue
    data_download(var,date_str)

In [ ]:
date='20231201'
var='Z'
for i in range(61+366+92):
    date_following = pd.to_datetime(date) + pd.Timedelta(days=i)
    date_str = date_following.strftime('%Y%m%d')
    if os.path.exists(r'E:\data\ERA5.daily.1x1\\'+var+'\\Hourly\\'+var+date_str+'.nc') == True:
        continue
    data_download(var,date_str)

In [ ]:
var='T2m'
years = [ str(id1) for id1 in range(2023,2024) ]
months =[ '%02d' % id2 for id2 in range(12,13) ]
for yr in years:
    for mn in months:
        list=glob.glob(r'E:\data\ERA5.daily.1x1\\'+var+'\\Hourly\\'+var+yr+mn+'*.nc')
        list.sort()
        data=[]
        for i in range(len(list)):
            data_sa=(xr.open_dataset(list[i])[var.lower()].transpose('valid_time','latitude','longitude'))
            data.append(data_sa)
        dataal=xr.concat(data,dim='valid_time')
        data_daily=dataal.rolling(valid_time=4).mean().sel(valid_time=dataal.valid_time[3::4])
        start_dt = f"{yr}-{mn}-01"
        end_dt = pd.Timestamp(start_dt) + pd.offsets.MonthEnd(1)
        date_var=pd.date_range(start_dt, end_dt, freq='D').to_numpy()
        data=xr.DataArray(data_daily,coords=[date_var,dataal.pressure_level,dataal.latitude,dataal.longitude],dims=['initial_time0_hours','g0_lat_1','g0_lon_2'],name=var)
        data.to_netcdf('E:\\data\\ERA5.daily.1x1\\'+var+'\\'+var+'.'+yr+mn+'.nc')

In [ ]:
var='T'
years = [ str(id1) for id1 in range(2024,2026) ]
months =[ '%02d' % id2 for id2 in range(1,13) ]
for yr in years:
    for mn in months:
        list=glob.glob(r'E:\data\ERA5.daily.1x1\\'+var+'\\Hourly\\'+var+yr+mn+'*.nc')
        list.sort()
        data=[]
        for i in range(len(list)):
            data_sa=(xr.open_dataset(list[i])[var.lower()].transpose('valid_time','pressure_level','latitude','longitude'))[:,:,::-1,:].sortby('pressure_level')
            data.append(data_sa)
        dataal=xr.concat(data,dim='valid_time')
        data_daily=dataal.rolling(valid_time=4).mean().sel(valid_time=dataal.valid_time[3::4])
        start_dt = f"{yr}-{mn}-01"
        end_dt = pd.Timestamp(start_dt) + pd.offsets.MonthEnd(1)
        date_var=pd.date_range(start_dt, end_dt, freq='D').to_numpy()
        data=xr.DataArray(data_daily,coords=[date_var,dataal.pressure_level,dataal.latitude,dataal.longitude],dims=['initial_time0_hours','lv_ISBL1','g0_lat_2','g0_lon_3'],name=var)
        data.to_netcdf('E:\\data\\ERA5.daily.1x1\\'+var+'\\'+var+'.'+yr+mn+'.nc')

In [5]:
var='Z'
years = [ str(id1) for id1 in range(2024,2026) ]
months =[ '%02d' % id2 for id2 in range(1,13) ]
for yr in years:
    for mn in months:
        list=glob.glob(r'E:\data\ERA5.daily.1x1\\'+var+'\\Hourly\\'+var+yr+mn+'*.nc')
        list.sort()
        data=[]
        for i in range(len(list)):
            data_sa=(xr.open_dataset(list[i])[var.lower()].transpose('valid_time','pressure_level','latitude','longitude'))[:,:,::-1,:].sortby('pressure_level')
            data.append(data_sa)
        dataal=xr.concat(data,dim='valid_time')
        data_daily=dataal.rolling(valid_time=4).mean().sel(valid_time=dataal.valid_time[3::4])
        start_dt = f"{yr}-{mn}-01"
        end_dt = pd.Timestamp(start_dt) + pd.offsets.MonthEnd(1)
        date_var=pd.date_range(start_dt, end_dt, freq='D').to_numpy()
        data=xr.DataArray(data_daily,coords=[date_var,dataal.pressure_level,dataal.latitude,dataal.longitude],dims=['initial_time0_hours','lv_ISBL1','g0_lat_2','g0_lon_3'],name=var)
        data.to_netcdf('E:\\data\\ERA5.daily.1x1\\'+var+'\\'+var+'.'+yr+mn+'.nc')

ValueError: must supply at least one object to concatenate